In [ ]:
import logging
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper

i2c_gui.__no_connect__ = False

port: str = "/dev/tty.usbmodem000613421"

logger = logging.getLogger("Script_Logger")

Script_Helper = i2c_gui.ScriptHelper(logger)

## USB ISS connection
conn = i2c_gui.Connection_Controller(Script_Helper)
conn.connection_type = "USB-ISS"
conn.handle: USB_ISS_Helper
conn.handle.port = port
conn.handle.clk = 100

conn.connect()

In [ ]:
chip = i2c_gui.chips.AD5593R_Chip(Script_Helper, conn)
chip.config_i2c_address(0x10)

# ADC Functionality

## Choose Enabled ADC channels

In [ ]:
adc_channels = {
    0: True,
    1: True,
    8: True,  # Channel 8 is temperature
}

for i in range(9):  # Channel 8 is temperature
    if i not in adc_channels:
        adc_channels[i] = False
    elif adc_channels[i] is not True:
        adc_channels[i] = False

## Choose Enabled DAC channels
*(for testing, do not enable the same pin for ADC and DAC for data taking)*

In [ ]:
dac_channels = {
    0: True,
    1: True,
}

for i in range(8):
    if i not in dac_channels:
        dac_channels[i] = False
    elif dac_channels[i] is not True:
        dac_channels[i] = False

## Choose VRef Source

In [ ]:
internal_vref = True  #  Set to False when using the SMA interface PCB

## Configure device

### Reset Device

In [ ]:
handle = chip.get_display_var("AD5593R", "Config_RD", "SW_RESET")
handle.set(0b10110101100)
chip.write_register("AD5593R", "Config_RD", "SW_RESET", write_check=False, no_message=True)
del handle

### Get Handles

In [ ]:
registers = [
    "ADC_SEQ",
    "GEN_CTRL_REG",
    "ADC_CONFIG",
    "DAC_CONFIG",
    "PULLDWN_CONFIG",
    "LDAC_MODE",
    "PD_REF_CTRL",
             ]

handle = {}
for name in registers:
    handle[name] = chip.get_display_var("AD5593R", "Config_RD", name)

### Read Register Values

In [ ]:
for name in registers:
    chip.read_register("AD5593R", "Config_RD", name, no_message=True)

### Enable VRef if needed

In [ ]:
value = int(handle["PD_REF_CTRL"].get(), 0)
if internal_vref:
    value = value | 0b0000_0010_0000_0000
else:
    value = value & 0b1111_1101_1111_1111

handle["PD_REF_CTRL"].set(value)
del value
chip.write_register("AD5593R", "Config_RD", "PD_REF_CTRL")

### Set DAC Mode

In [ ]:
handle["LDAC_MODE"].set(0x0000)  # alternative is 0x0001, but in this mode you need to write LDAC_MODE with 0x0002 to update the DAC output
chip.write_register("AD5593R", "Config_RD", "LDAC_MODE")

for i in range(8):
    tmp_handle = chip.get_display_var("AD5593R", "DAC_RD", f"DAC{i}")
    tmp_handle.set(0)
    chip.write_register("AD5593R", "DAC_RD", f"DAC{i}", write_check=False, no_message=True)
    del tmp_handle

### Configure ADC Mode

In [ ]:
# Use ADC buffer if not using the SMA adaptor PCB
# Disable ADC buffer when using the SMA adaptor PCB

value = 0b000000_0_0_0_0_0_0_0000  #  Disable ADC buffer
value = 0b000000_0_1_0_0_0_0_0000  #  Enable ADC buffer and keep it always powered
#value = 0b000000_1_1_0_0_0_0_0000  #  Enable ADC buffer and only power during conversion

handle["GEN_CTRL_REG"].set(value)
chip.write_register("AD5593R", "Config_RD", "GEN_CTRL_REG")

### Enable pins for ADC/DAC
*Pins that are enabled for DAC/ADC should have pulldown disabled*

In [ ]:
adc_reg = 0
dac_reg = 0
pulldown_reg = 0x00ff
num_measurements = 0

for pin in range(8):
    if dac_channels[pin]:  #  Enable DAC pins
        dac_reg = (dac_reg | (1 << pin))
        pulldown_reg = (pulldown_reg & (0xffff - (1 << pin)))
    if adc_channels[pin]:  #  Enable ADC pins
        adc_reg = (adc_reg | (1 << pin))
        pulldown_reg = (pulldown_reg & (0xffff - (1 << pin)))
        num_measurements += 1

handle["DAC_CONFIG"].set(dac_reg)
handle["ADC_CONFIG"].set(adc_reg)
handle["PULLDWN_CONFIG"].set(pulldown_reg)

if adc_channels[8]:  #  Enable temperature in the ADC sequence if it is chosen
    adc_reg = adc_reg | 0x100
    num_measurements += 1
adc_sequence = adc_reg

chip.write_register("AD5593R", "Config_RD", "DAC_CONFIG")
chip.write_register("AD5593R", "Config_RD", "ADC_CONFIG")
chip.write_register("AD5593R", "Config_RD", "PULLDWN_CONFIG")

## SET DAC0 to a specific voltage

In [ ]:
chip.get_display_var("AD5593R", "DAC_RD", f"DAC0").set(0x800)
chip.write_register("AD5593R", "DAC_RD", f"DAC0", write_check=False, no_message=True)


## Do a single ADC conversion

In [ ]:
measurements = chip.read_adc_results(adc_sequence, num_measurements)

print("Measurements:")
for m in measurements:
    print(f' - {m:#018b}')

#handle["ADC_SEQ"].set(adc_reg)
#chip.write_register("AD5593R", "Config_RD", "ADC_SEQ")

In [ ]:
from scripts.read_adc import ADCMeasurements
from pathlib import Path

port: str = "/dev/ttyACM4"

link = ADCMeasurements(Path("./"), 5, port, vref = 2.5, i2c_address = 0x10, internal_vref = True)

link.add_temperature()
link.add_adc_pin(0)
link.add_adc_pin(1)
link.add_adc_pin(2)
link.add_adc_pin(3)
link.add_adc_pin(4)
link.add_adc_pin(5)
link.add_adc_pin(6)
link.add_adc_pin(7)

link.add_dac_pin(0)
link.add_dac_pin(1)
link.add_dac_pin(2)
link.add_dac_pin(3)
link.add_dac_pin(4)
link.add_dac_pin(5)
link.add_dac_pin(6)
link.add_dac_pin(7)

link.configure()

In [ ]:
link.set_dac_value(0x000)
link.do_measurement()

In [ ]:
link.set_dac_value(0x800)
link.do_measurement()

In [ ]:
link.set_dac_value(0xfff)
link.do_measurement()